## Actual Working Script

In [1]:
!pip install gspread
!pip install oauth2client
!pip install -U snowflake-connector-python
!pip install -U snowflake-snowpark-python
!pip install --upgrade psycopg2
!pip install -U sqlalchemy
!pip install df2gspread

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 89.6 MB/s eta 0:00:00
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 3.1.1
    Uninstalling cloudpickle-3.1.1:
      Successfully uninstalled cloudpickle-3.1.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [snowflake-snowpark-python]e-snowpark-python]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 68.7 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.41
    Uninstalling SQLAlchemy-2.0.41:
      Successfully uninstalled SQLAlchemy-2.0.41
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'df2gspread' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the 

In [2]:
from common_functions import ret_metabase
from datetime import datetime, timedelta
import datetime as dt
import time
import os
import boto3
import base64
from botocore.exceptions import ClientError
import json
import requests
from pathlib import Path
from io import StringIO
import pandas as pd
import sqlalchemy
import psycopg2
import numpy as np
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import os
import requests
import json
import pandas as pd
import numpy as np
import re
import datetime
import seaborn as sns
from io import StringIO, BytesIO
import sys
from datetime import datetime



In [3]:
import requests

def address_details(lat, lon):
    # Construct the API URL
    url = f"https://nominatim.openstreetmap.org/reverse?format=jsonv2&lat={lat}&lon={lon}"
    
    # Headers to avoid being blocked and to force English response
    headers = {
        'User-Agent': 'Mozilla/5.0 (compatible; MyGeocodingApp/1.0; +http://example.com/my-geocoding-app)',
        'Accept-Language': 'en'  # Force response in English
    }
    
    # Make the API request
    # Check if the request was successful
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            display_name = data.get("display_name")
            name = data.get("name")
            road = data.get("address", {}).get("road")
            suburb = data.get("address", {}).get("suburb")
            city = data.get("address", {}).get("city") or data.get("address", {}).get("town") or data.get("address", {}).get("village")
            state = data.get("address", {}).get("state")
            country = data.get("address", {}).get("country")
            return display_name, name, road, suburb, city, state, country
        else:
            print(f"API error: status code {response.status_code}")
            return None, None, None, None, None, None, None
    except Exception as e:
        print(f"Exception during API call: {e}")
        return None, None, None, None, None, None, None

# Morocco

In [4]:
df = ret_metabase("Morocco" , 18371)

/home/ec2-user/service_account_key.json


In [30]:
initialize_env()

df_1 = pd.DataFrame(ret_metabase(os.environ["AFRICA_METABASE_URL"],os.environ["AFRICA_METABASE_USERNAME"],os.environ["AFRICA_METABASE_PASSWORD"], 18371))

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [5]:
df.head()

,MAIN_SYSTEM_ID,ID,LATITUDE,LONGITUDE
0,63022,62364,32.493311,-6.687155
1,62563,61908,34.010237,-5.025463
2,51838,36601,30.345239,-9.495779
3,48263,53368,33.447318,-5.224654
4,66692,65998,35.172550,-5.274645


In [31]:
df_1.head()

,main_system_id,id,latitude,longitude
0,62472,61817,30.896836,-6.894628
1,55670,55131,31.595410,-8.031751
2,62866,62209,35.237845,-3.936388
3,62087,61433,35.200145,-3.048387
4,63033,62375,29.957432,-7.554077


In [32]:
import pandas as pd
import requests
from time import sleep

# Initialize an empty list to store the updated data
updated_data = []
print(len(df_1))

# Process retailers in batches of 50
batch_size = 50


for start in range(0, len(df_1), batch_size):
    batch = df_1.iloc[start:start + batch_size]
    for index, row in batch.iterrows():
        main_system_id = row['main_system_id']
#         cash_in_id = row['cash_in_id']
        retailer_id =  row['id']
        lat = row['latitude']
        lon = row['longitude']
        try:
            display_name, name, road, suburb, city, state, country = address_details(lat, lon)
        except Exception as e:
            print(f"Error processing retailer_id {retailer_id}: {e}")
            display_name, name, road, suburb, city, state, country = [None] * 7
        
        updated_data.append([
             main_system_id, retailer_id, display_name, city, state, country
        ])
        
#         print(updated_data)
#         sleep(1)

    df_temp = pd.DataFrame(updated_data, columns=[
       'main_system_id','retailer_id', 'address', 'district', 'state', 'country'])
#     df_temp.to_excel('retailers_partial_output.xlsx', index=False)
    
    sleep(1)

# --- Step 4: Final save ---
df_updated = pd.DataFrame(updated_data, columns=['main_system_id','retailer_id', 'address', 'district', 'state', 'country'])
# df_updated.to_excel('retailers_with_regions.xlsx', index=False)
# print("✅ Exported final file: retailers_with_regions.xlsx")

8652
[[62472.0, 61817.0, "Riad Bouchedor, RN9, Douar Tajda, Tabounte, Tarmigt, caïdat de Ahl Ouarzazate, cercle d'Ouarzazate, Ouarzazate Province, Drâa-Tafilalet, 45000, Morocco", 'Tarmigt', None, 'Morocco']]
[[62472.0, 61817.0, "Riad Bouchedor, RN9, Douar Tajda, Tabounte, Tarmigt, caïdat de Ahl Ouarzazate, cercle d'Ouarzazate, Ouarzazate Province, Drâa-Tafilalet, 45000, Morocco", 'Tarmigt', None, 'Morocco'], [55670.0, 55131.0, 'Arrondissement de Ménara مقاطعة المنارة, Marrakesh, Pachalik de Marrakech, Marrakesh Prefecture, Marrakech-Safi, 40054, Morocco', 'Marrakesh', None, 'Morocco']]


KeyboardInterrupt: 

# Egypt

tablename: reatiler_addresses_v2 
schema name: fintech 
country: EG 
df: df_updated 

In [33]:
initialize_env()

df_2 = pd.DataFrame(ret_metabase(os.environ["EGYPT_METABASE_URL"],os.environ["EGYPT_METABASE_USERNAME"],os.environ["EGYPT_METABASE_PASSWORD"], 63420))

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [34]:
df_2.head()

,main_system_id,id,latitude,longitude
0,508388,1216203,30.879824,32.371820
1,745544,1944204,30.999676,30.668432
2,677044,1833185,25.708316,32.644156
3,583356,1738669,30.940219,31.245829
4,918721,2242866,30.746532,31.272261


In [36]:
# Initialize an empty list to store the updated data
updated_data_2 = []
print(len(df_2))

# Process retailers in batches of 50
batch_size2 = 50


for start in range(0, len(df_2), batch_size2):
    batch2 = df_2.iloc[start:start + batch_size2]
    for index, row in batch2.iterrows():
        main_system_id2 = row['main_system_id']
        retailer_id2 =  row['id']
        lat2 = row['latitude']
        lon2 = row['longitude']
        try:
            display_name2, name2, road2, suburb2, city2, state2, country2 = address_details(lat2, lon2)
        except Exception as e:
            print(f"Error processing retailer_id {retailer_id2}: {e}")
            display_name2, name2, road2, suburb2, city2, state2, country2 = [None] * 7
        
        updated_data_2.append([
            main_system_id2, retailer_id2, display_name2, city2, state2, country2 ])
        
        print(updated_data_2)
#         sleep(1)

    df_temp_2 = pd.DataFrame(updated_data_2, columns=[
        'main_system_id','retailer_id', 'address', 'district', 'state', 'country'])
#     df_temp.to_excel('retailers_partial_output.xlsx', index=False)
    
    sleep(1)

# --- Step 4: Final save ---
df_updated_2 = pd.DataFrame(updated_data_2, columns=[
    'main_system_id','retailer_id', 'address', 'district', 'state', 'country'
])
# df_updated_2.to_excel('retailers_with_regions.xlsx', index=False)
# print("✅ Exported final file: retailers_with_regions.xlsx")

52666
[[508388.0, 1216203.0, 'Ras Sedr, Al Qantra Sharq Road, Qantara East, Port Said, Egypt', 'Qantara East', 'Port Said', 'Egypt']]


KeyboardInterrupt: 